In [1]:
%matplotlib widget

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from glob import glob
from os import getcwd, path as path_lib
import math

log_path = path_lib.join(getcwd(), "ConSteer")
scene_path = path_lib.join(getcwd().split("\\")[0].upper()+"\\",*getcwd().split("\\")[1:-1],"Project-Code","Scenarios","All_2Cop.csv")
# for scene_path use scene.csv not scene_swarm_position.csv file path
# for log_path link to drone log file directory that matches with the scene

In [2]:
drone_ids = [*range(50,52)]
files = glob(path_lib.join(log_path,"*.log"))

len(files) == len(drone_ids)

True

In [3]:
drone_protocol = dict()

for d_id,file_path in zip(drone_ids,files):
    drone_protocol[d_id] = dict()
    with open(file_path,"r") as file:
        lines = file.readlines()
        for line in lines[1:-1]:
            key = float(line.split("]")[0].strip("[").strip())
            pos = [float(coord.strip()) for coord in line.split("pos:")[-1].split("/")[0].split(",")]
            drone_protocol[d_id][key] = pos

print(drone_protocol.keys())
for key in drone_protocol.keys():
    print("-> protocol from drone "+str(key)+": ")
    print("   first_timestamp:",list(drone_protocol[key].keys())[0])
    print("   first_pos:",drone_protocol[key][list(drone_protocol[key].keys())[0]])
    print("   timestamp_amount:",len(drone_protocol[key].keys()))


dict_keys([50, 51])
-> protocol from drone 50: 
   first_timestamp: 0.02931
   first_pos: [52.13591652, 11.64344738]
   timestamp_amount: 317
-> protocol from drone 51: 
   first_timestamp: 0.02931
   first_pos: [52.13741233, 11.64344738]
   timestamp_amount: 317


In [4]:
scene = dict()

with open(scene_path, "r") as file:
    lines = file.readlines()
    for line in lines[2:]:
        vals = [val.strip() for val in line.split(",")]
        scene[vals[-1]] = scene.get(vals[-1], [])
        scene[vals[-1]].append([round(float(vals[0]),8),round(float(vals[1]),8)])

print(scene.keys())
for key in scene.keys():
    print("-> "+str(key)+"s in scene: ")
    print("   first_"+str(key)+":",scene[key][0])
    print("   "+str(key)+"_amount:",len(scene[key]))

dict_keys(['att', 'rep', 'spawn'])
-> atts in scene: 
   first_att: [52.13716576, 11.64242715]
   att_amount: 1
-> reps in scene: 
   first_rep: [52.13741576, 11.64354715]
   rep_amount: 3
-> spawns in scene: 
   first_spawn: [52.13591576, 11.64344715]
   spawn_amount: 2


GOAL Achievement Analysis

In [5]:
reached = {}
for timestamp in drone_protocol[list(drone_protocol.keys())[0]].keys():
    for d_id in drone_protocol.keys():
        for att_id, att in enumerate(scene["att"]):
            if abs(drone_protocol[d_id][timestamp][0] - att[0]) < 0.00001 \
            and abs(drone_protocol[d_id][timestamp][1] - att[1]) < 0.00001 \
            and not reached.get(str(d_id)+"->"+str(att_id), None):
                reached[str(d_id)+"->"+str(att_id)] = timestamp

for key in reached.keys():
    print("Drone",key.split("->")[0],"reached ATT_Point",key.split("->")[1],"approximately",
    str(reached[key])+"sec after simulation start.")

Drone 51 reached ATT_Point 0 approximately 103.669472sec after simulation start.


DANGER Avoidance Analysis

In [6]:
min_dist = {}
for timestamp in drone_protocol[list(drone_protocol.keys())[0]].keys():
    for d_id in drone_protocol.keys():
        for rep_id, rep in enumerate(scene["rep"]):
            dist = round(math.sqrt((drone_protocol[d_id][timestamp][0] - rep[0])**2 + 
                   (drone_protocol[d_id][timestamp][1] - rep[1])**2),12)
            if not min_dist.get(str(d_id)+"->"+str(rep_id), None) \
            or dist < min_dist[str(d_id)+"->"+str(rep_id)][1]:
                min_dist[str(d_id)+"->"+str(rep_id)] = [timestamp, dist]

for key in min_dist.keys():
    print("Smallest distance between Drone",key.split("->")[0],"and REP_Point",key.split("->")[1],
    "was {0:.12f}m and occured".format(min_dist[key][1]),
    "{0:.6f}sec after simulation start.".format(min_dist[key][0]))

Smallest distance between Drone 50 and REP_Point 0 was 0.000916832319m and occured 73.374657sec after simulation start.
Smallest distance between Drone 50 and REP_Point 1 was 0.000813987257m and occured 81.745590sec after simulation start.
Smallest distance between Drone 50 and REP_Point 2 was 0.000779376644m and occured 75.766350sec after simulation start.
Smallest distance between Drone 51 and REP_Point 0 was 0.000098969455m and occured 28.729662sec after simulation start.
Smallest distance between Drone 51 and REP_Point 1 was 0.000075050656m and occured 35.107517sec after simulation start.
Smallest distance between Drone 51 and REP_Point 2 was 0.000053835769m and occured 33.513054sec after simulation start.
